In [12]:
import sqlite3
import numpy as np

In [13]:
def fetchTrainingData(db, sqlCmd):
    try:
        conn = sqlite3.connect(db)
        cursor = conn.cursor()
        cursor.execute(sqlCmd)
        result = cursor.fetchall()
        conn.commit()
        cursor.close()
        return result

    except sqlite3.Error as error:
        print("Failed to run sqlite command", error)
    finally:
        if (conn):
            conn.close()

In [14]:
db_path = 'db/categorised_swecris.sqlite'

In [15]:
init = False
for ix in range(2,18):
    sqlCmd = f'''
        SELECT abstract, Formas1 AS class FROM clean_swecris
            WHERE class == '{ix}' ORDER BY RANDOM() LIMIT 99
        '''
    result = np.array(fetchTrainingData(db_path, sqlCmd))
    if (not init and result.size > 0):
        training_data_Formas = np.copy(result)
        init = True
    elif (result.size > 0):
        training_data_Formas = np.vstack((training_data_Formas, result))

In [16]:
init = False
for ix in range(2,18):
    sqlCmd = f'''
        SELECT abstract, Self1 AS class FROM clean_swecris 
            WHERE Self1 == '{ix}' ORDER BY RANDOM()
            LIMIT
                CASE 
                    WHEN (99 - (SELECT count(Formas1) FROM clean_swecris WHERE Formas1 == '{ix}')) > 0
                        THEN (SELECT 99 - count(Formas1) FROM clean_swecris WHERE Formas1 == '{ix}')
                    ELSE 0
                END;
        '''
    result = np.array(fetchTrainingData(db_path, sqlCmd))
    if (not init and result.size > 0):
        training_data_Vinnova = np.copy(result)
        init = True
    elif (result.size > 0):
        training_data_Vinnova = np.vstack((training_data_Vinnova, result))
        print(ix)

4
5
6
7
8
9
10
11
13
14
16
17


In [17]:
init = False
for ix in range(2,17):
    sqlCmd = f'''
        SELECT abstract, goals AS class FROM clean_goals
            WHERE class == '{ix}';
        '''
    result = np.array(fetchTrainingData(db_path, sqlCmd))
    if (not init and result.size > 0):
        training_data_Goals = np.copy(result)
        init = True
    else:
        training_data_Goals = np.vstack((training_data_Goals, result))

In [18]:
training_data = np.vstack((training_data_Goals, training_data_Formas, training_data_Vinnova))

In [19]:
import pandas as pd

In [20]:
training_data_df = pd.DataFrame(data=training_data[:,:], columns=['abstract', 'class'])

In [21]:
training_data_df

,abstract,class
0,ingen hung hung vara en främst dödsorsak i vär...,2
1,god hälsa och välbefinnande god hälsa vara en ...,3
2,god utbildning för alla utbildning vara en gru...,4
3,jämställdhet jämställdhet mellan kvinna och ma...,5
4,ren vatten och sanitet för alla vatten vara en...,6
...,...,...
1528,mcflairs vara en startup och vi få chans att...,17
1529,utveckla en hållbar och mobil tältplattform ...,17
1530,denna ansökan avse en delfinansiering av res...,17
1531,ha utreda förutsättning för en testbädd i sa...,17
